In [ ]:
from pathlib2 import Path
from tqdm.notebook import tqdm

import numpy as np
import seaborn as sns
import scipy.io as spio

import matplotlib.pyplot as plt
import matplotlib.animation as animation

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams["animation.embed_limit"] = 512
plt.rcParams['figure.dpi'] = 150  
%matplotlib inline

## Load data

In [ ]:
root = Path("/media/agjvc_rad3/_TESTKOLLEKTIV/Daten")

In [ ]:
cases = list(filter(lambda case: len(list(case.glob("Series*"))) != 0, root.glob("*/*")))
cases_len = len(cases)

series = list(root.glob("*/*/Series*/dicoms.mat"))
series_len = len(series)

print(f"Cases: {cases_len}, Series: {series_len}")

## Visualisation

### Series

In [ ]:
case_idx = 0
assert(case_idx < cases_len)

In [ ]:
case_series = list(cases[case_idx].glob("Series*/dicoms.mat"))
case_series_len = len(case_series)
case_series_data = [(spio.loadmat(path)["dcm"]) for path in tqdm(case_series)]
assert(len(case_series_data) > 0)

case_sequence_len = case_series_data[0].shape[0]
print(f"Series: {case_series_len}, Sequence-length: {case_sequence_len}")

In [ ]:
case_sequence_idx = 35
assert(case_sequence_idx < case_sequence_len)

In [ ]:
fig, ax = plt.subplots(1, case_series_len, figsize=(case_series_len, 1), dpi=300)

for i, d in enumerate(case_series_data):
    ax[i].imshow(d[case_sequence_idx], interpolation="nearest", cmap="gray")
    ax[i].set_title(f"Case {case_idx} Series {i}", fontsize=4)
    ax[i].set_xticks([])
    ax[i].set_yticks([])

plt.show()
plt.close()


### Frames

In [ ]:
case_series_idx = 3
assert(case_series_idx < case_series_len)
image_data = case_series_data[case_series_idx]

In [ ]:
fig, ax = plt.subplots()

im = ax.imshow(image_data[0], animated=True, cmap="gray")
def animate(i):
    im.set_data(image_data[i])
    return [im]

animation.FuncAnimation(fig, animate, frames=image_data.shape[0])

## Analysis

In [ ]:
def calculate_histogram(image):
  return np.histogram(image.flatten(), bins=256)[0]

def calculate_intensity_stats(image):
  flat_image = image.flatten()
  mean = np.mean(flat_image)
  std = np.std(flat_image)
  var = np.var(flat_image)
  return mean, std, var

def analyse_series(series):
    stats = {}
    stats["histogram"] = calculate_histogram(series)
    stats["mean"], stats["std"], stats["var"] = calculate_intensity_stats(series)
    return stats

def load_series(path):
    return spio.loadmat(path)["dcm"]

In [ ]:
series_stats = [analyse_series(load_series(s)) for s in tqdm(series)]

In [ ]:
rows, cols = 3, 5
fig, ax = plt.subplots(rows, cols,figsize=(16,8))
np.random.seed(42)
for i in range(rows):
    for j in range(cols):
        idx = np.random.randint(0, len(series_stats))
        ax[i][j].plot(series_stats[idx]["histogram"], "-", lw=1)
        ax[i][j].set_title(idx)
        ax[i][j].set_yscale("log")
        ax[i][j].set_ylabel("Frequency")
        ax[i][j].set_yticks([1e2,1e4,1e6])
        ax[i][j].set_xlabel("Intensity (bin_size=256)")
plt.suptitle("Histograms")
plt.tight_layout()
plt.savefig("histograms.png")
plt.show()
plt.close()

In [ ]:
means = [stats['mean'] for stats in series_stats]
stds = [stats['std'] for stats in series_stats]
variances = [stats['var'] for stats in series_stats]

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(16,4))

ax[0].hist(means, bins=128, density=True)
ax[0].set_title("Mean")
ax[1].hist(stds, bins=128, density=True)
ax[1].set_title("Standard deviation")
ax[2].hist(variances, bins=128, density=True)
ax[2].set_title("Variance")

plt.savefig("distributions.png")
plt.show()
plt.close()